# Prototype python script for fetching and averaging ACCESS data

The goal is to be able to create NetCDF files of time-averaged ocean-transport states (`umo`, `vmo`, `uo`, `vo`, and `mlotst`) from ACCESS runs.

Eventually, this notebook will be turned into a function with these inputs:
- model
- experiment
- ensemble member
- time period
that a script can put in a loop to generate lots of datasets.

Down the road, a Julia script using [OceanTransportMatrixBuilder.jl](https://github.com/TMIP-code/OceanTransportMatrixBuilder.jl) can grab the time-averaged data from these files and generate the transport matrices.

Note that this notebook is specific to Gadi at NCI. It may require access to at least one of the following projects (so make sure you add them all in your interactive ARE job submission):
```
gdata/gh0+gdata/xv83+gdata/oi10+gdata/dk92+gdata/hh5+gdata/rr3+gdata/al33+gdata/fs38
```

Warning: No promises made, this is work in progress!

## 1. Load packages

In [1]:
# Ignore warnings
from os import environ
environ["PYTHONWARNINGS"] = "ignore"

In [2]:
# Import makedirs to create directories where I write new files
from os import makedirs

In [3]:
# Load dask
from dask.distributed import Client

# Load intake and cosima cookbook
import intake
import cosima_cookbook as cc

# Load xarray for N-dimensional arrays
import xarray as xr

# Load xesmf for regridding
import xesmf as xe

# Load datetime to deal with time formats
import datetime

# Load numpy for numbers!
import numpy as np

# Load xmip for preprocessing (trying to get consistent metadata for making matrices down the road)
from xmip.preprocessing import combined_preprocessing

# Load pandas for DataFrame manipulations
import pandas as pd

## 2. Define some functions

(to avoid too much boilerplate code)

In [4]:
########## functions ##########
print("Defining functions")

def time_window_strings(year_start, num_years, time_type = datetime.datetime):
    """
    return strings for start_time and end_time
    """
    # start_time is first second of year_start
    start_time = time_type(year_start, 1, 1, 0, 0, 0)
    # end_time is last second of last_year
    end_time = time_type(year_start + num_years - 1, 12, 31, 23, 59, 59)

    # Return the weighted average
    return start_time, end_time

def find_latest_version(cat):
    """
    find latest version of selected data
    """
    sorted_versions = cat.df.version.to_list()
    sorted_versions.sort()
    latest_version = sorted_versions[-1]
    return latest_version

def select_latest_cat(cat, **kwargs):
    """
    search latest version of selected data
    """
    selectedcat = cat.search(**kwargs)
    latestselectedcat = selectedcat.search(version=find_latest_version(selectedcat))
    return latestselectedcat

def select_latest_data(cat, xarray_open_kwargs, **kwargs):
    latestselectedcat = select_latest_cat(cat, **kwargs)
    xarray_combine_by_coords_kwargs=dict(
        compat="override",
        data_vars="minimal",
        coords="minimal"
    )
    datadask = latestselectedcat.to_dask(
        xarray_open_kwargs=xarray_open_kwargs,
        xarray_combine_by_coords_kwargs=xarray_combine_by_coords_kwargs,
        parallel=True
    )
    return datadask



Defining functions


## 3. Select the model, experiment, ensemble, and time window

On NCI, the catalog depends not only on the CMIP version but also on the model: For some reason that eludes me, some of the Australian data (including ACCESS1.3, ACCESS-ESM1.5, and ACCESS-CM2) lives in its own ***separate*** catalog. So I need to define these first 

In [5]:
# Comment/uncomment
model = "ACCESS-ESM1-5"

In [6]:
# Load catalog
cat_str = "/g/data/dk92/catalog/v2/esm/cmip6-fs38/catalog.json"

In [7]:
# The catalog
cat = intake.open_esm_datastore(cat_str)
cat

,unique
path,1054133
file_type,2
realm,7
frequency,10
table_id,24
project_id,1
institution_id,3
source_id,4
experiment_id,52
member_id,80


A little detour to list all the models in the catalog:

In [8]:
models = np.sort(cat.search(realm = 'ocean').df.source_id.unique())
print(*models, sep = "\n")

ACCESS-CM2
ACCESS-ESM1-5
ACCESS-OM2
ACCESS-OM2-025


The creation of a matrix can only work if the following set of variables is available:
- mass transports (`umo` and `vmo`)
- mixed-layer depth (`mlotst`)

Alternatively, we can use `umo` and `vmo` (in kg/s) can be replaced by `uo` and `vo` (m/s). However, the conversion to mass transport requires the grid-cell volume (`volcello`), grid-cell areas (from vertices and `thkcello`), and density (no variable so I guess constant will do).

So the notebook here will create all the files for transport, if available: `umo`, `vmo`, `uo`, `vo`, `mlotst`.

In [9]:
experiment = "historical"

List of members

In [10]:
members = np.sort(cat.search(source_id = model, realm = 'ocean').df.member_id.unique())
print(*members, sep = "\n")

r10i1p1f1
r10i1p1f2
r11i1p1f1
r11i1p1f2
r12i1p1f1
r12i1p1f2
r13i1p1f1
r13i1p1f2
r14i1p1f1
r14i1p1f2
r15i1p1f1
r15i1p1f2
r16i1p1f1
r16i1p1f2
r17i1p1f1
r17i1p1f2
r18i1p1f1
r18i1p1f2
r19i1p1f1
r19i1p1f2
r1i1p1f1
r1i1p1f2
r20i1p1f1
r20i1p1f2
r21i1p1f1
r21i1p1f2
r22i1p1f1
r22i1p1f2
r23i1p1f1
r23i1p1f2
r24i1p1f1
r24i1p1f2
r25i1p1f1
r25i1p1f2
r26i1p1f1
r26i1p1f2
r27i1p1f1
r27i1p1f2
r28i1p1f1
r28i1p1f2
r29i1p1f1
r29i1p1f2
r2i1p1f1
r2i1p1f2
r30i1p1f1
r30i1p1f2
r31i1p1f1
r31i1p1f2
r32i1p1f1
r32i1p1f2
r33i1p1f1
r33i1p1f2
r34i1p1f1
r34i1p1f2
r35i1p1f1
r35i1p1f2
r36i1p1f1
r36i1p1f2
r37i1p1f1
r37i1p1f2
r38i1p1f1
r38i1p1f2
r39i1p1f1
r39i1p1f2
r3i1p1f1
r3i1p1f2
r40i1p1f1
r40i1p1f2
r4i1p1f1
r4i1p1f2
r5i1p1f1
r5i1p1f2
r6i1p1f1
r6i1p1f2
r7i1p1f1
r7i1p1f2
r8i1p1f1
r8i1p1f2
r9i1p1f1
r9i1p1f2


In [11]:
def list_models_and_members_that_have(cat, variables):
    """
    find the list of models and their members that have all the variables: 'uo', 'vo', and 'mlotst'.
    """
    # Step 1: Filter the dataframe to include only the specified variables
    filtered_df = cat.search(variable_id = variables).df
    
    # Step 2: Group by 'model' and 'member'
    grouped = filtered_df.groupby(['source_id', 'member_id'])
    
    # Step 3: Find groups that contain all three variables
    valid_groups = grouped.filter(lambda x: set(variables).issubset(set(x['variable_id'])))
    
    # Step 4: Get the list of models and their members
    result = valid_groups[['source_id', 'member_id']].drop_duplicates().reset_index(drop=True)
    
    # Step 5: Sort the result by model
    result_sorted = result.sort_values(by='source_id')

    # Setp 6: Regroup by model
    grouped_result_sorted = result_sorted.groupby('source_id')
    
    return grouped_result_sorted.apply(display)

In [12]:
def summary_variable_availability(df):

    # Step 1: Filter the dataframe to include only the specified variables
    filtered_df_1 = df[df['variable_id'].isin(['umo', 'vmo'])]
    filtered_df_2 = df[df['variable_id'].isin(['uo', 'vo'])]
    filtered_df_3 = df[df['variable_id'].isin(['mlotst'])]
    filtered_df_4 = df[df['variable_id'].isin(['mlotstmax'])]
    
    # Step 2: Group by 'source_id' and 'member_id'
    grouped_1 = filtered_df_1.groupby(['experiment_id', 'source_id', 'member_id'])
    grouped_2 = filtered_df_2.groupby(['experiment_id', 'source_id', 'member_id'])
    grouped_3 = filtered_df_3.groupby(['experiment_id', 'source_id', 'member_id'])
    grouped_4 = filtered_df_4.groupby(['experiment_id', 'source_id', 'member_id'])
    
    # Step 3: Find groups that contain all the variables in each set
    valid_groups_1 = grouped_1.filter(lambda x: set(['umo', 'vmo']).issubset(set(x['variable_id'])))
    valid_groups_2 = grouped_2.filter(lambda x: set(['uo', 'vo']).issubset(set(x['variable_id'])))
    valid_groups_3 = grouped_3.filter(lambda x: set(['mlotst']).issubset(set(x['variable_id'])))
    valid_groups_4 = grouped_4.filter(lambda x: set(['mlotstmax']).issubset(set(x['variable_id'])))
    
    # Step 4: Get the list of source_id and their member_id for each set
    result_1 = valid_groups_1[['experiment_id', 'source_id', 'member_id']].drop_duplicates().reset_index(drop=True)
    result_2 = valid_groups_2[['experiment_id', 'source_id', 'member_id']].drop_duplicates().reset_index(drop=True)
    result_3 = valid_groups_3[['experiment_id', 'source_id', 'member_id']].drop_duplicates().reset_index(drop=True)
    result_4 = valid_groups_4[['experiment_id', 'source_id', 'member_id']].drop_duplicates().reset_index(drop=True)
    
    # Step 5: Group by 'source_id' and aggregate member_id into a list for each set
    final_result_1 = result_1.groupby(['experiment_id', 'source_id'])['member_id'].apply(list).reset_index()
    final_result_2 = result_2.groupby(['experiment_id', 'source_id'])['member_id'].apply(list).reset_index()
    final_result_3 = result_3.groupby(['experiment_id', 'source_id'])['member_id'].apply(list).reset_index()
    final_result_4 = result_4.groupby(['experiment_id', 'source_id'])['member_id'].apply(list).reset_index()
    
    # Step 6: Merge the results into a single dataframe
    merged_result_1 = pd.merge(final_result_1, final_result_2, on=['experiment_id', 'source_id'], how='outer', suffixes=('_umo_vmo', '_uo_vo'))
    merged_result_2 = pd.merge(merged_result_1, final_result_3, on=['experiment_id', 'source_id'], how='outer', suffixes=('', '_mlotst'))
    merged_result_3 = pd.merge(merged_result_2, final_result_4, on=['experiment_id', 'source_id'], how='outer', suffixes=('', '_mlotstmax'))

    final_restult = merged_result_3.sort_values(by='source_id')
    
    return final_restult

In [13]:
summary_variable_availability(cat.df)

,experiment_id,source_id,member_id_umo_vmo,member_id_uo_vo,member_id,member_id_mlotstmax
0,1pctCO2,ACCESS-CM2,[r1i1p1f1],[r1i1p1f1],[r1i1p1f1],NaN
54,ssp585,ACCESS-CM2,"[r1i1p1f1, r4i1p1f1, r6i1p1f1, r5i1p1f1, r3i1p...","[r6i1p1f1, r9i1p1f1, r1i1p1f1, r10i1p1f1, r2i1...","[r5i1p1f1, r8i1p1f1, r9i1p1f1, r4i1p1f1, r1i1p...",NaN
52,ssp534-over,ACCESS-CM2,[r1i1p1f1],[r1i1p1f1],[r1i1p1f1],NaN
49,ssp370,ACCESS-CM2,"[r10i1p1f1, r9i1p1f1, r5i1p1f1, r8i1p1f1, r1i1...","[r8i1p1f1, r5i1p1f1, r2i1p1f1, r6i1p1f1, r3i1p...","[r8i1p1f1, r10i1p1f1, r4i1p1f1, r7i1p1f1, r3i1...",NaN
43,ssp245,ACCESS-CM2,"[r10i1p1f1, r9i1p1f1, r2i1p1f1, r1i1p1f1, r6i1...","[r10i1p1f1, r9i1p1f1, r6i1p1f1, r2i1p1f1, r3i1...","[r10i1p1f1, r1i1p1f1, r8i1p1f1, r7i1p1f1, r5i1...",NaN
40,ssp126,ACCESS-CM2,"[r1i1p1f1, r10i1p1f1, r8i1p1f1, r6i1p1f1, r2i1...","[r10i1p1f1, r1i1p1f1, r9i1p1f1, r8i1p1f1, r5i1...","[r4i1p1f1, r1i1p1f1, r5i1p1f1, r2i1p1f1, r10i1...",NaN
38,piControl,ACCESS-CM2,[r1i1p1f1],[r1i1p1f1],[r1i1p1f1],NaN
31,historical,ACCESS-CM2,"[r10i1p1f1, r7i1p1f1, r9i1p1f1, r8i1p1f1, r6i1...","[r9i1p1f1, r10i1p1f1, r7i1p1f1, r8i1p1f1, r5i1...","[r10i1p1f1, r7i1p1f1, r8i1p1f1, r1i1p1f1, r4i1...",NaN
25,hist-aer,ACCESS-CM2,NaN,"[r3i1p1f1, r1i1p1f1, r2i1p1f1]","[r3i1p1f1, r1i1p1f1, r2i1p1f1]",NaN
23,hist-GHG,ACCESS-CM2,NaN,"[r1i1p1f1, r2i1p1f1, r3i1p1f1]","[r2i1p1f1, r1i1p1f1, r3i1p1f1]",NaN


In [14]:
model

'ACCESS-ESM1-5'

In [15]:
list_models_and_members_that_have(cat, ['umo', 'vmo', 'mlotst'])

,source_id,member_id
89,ACCESS-CM2,r3i1p1f1
88,ACCESS-CM2,r6i1p1f1
87,ACCESS-CM2,r2i1p1f1
86,ACCESS-CM2,r4i1p1f1
85,ACCESS-CM2,r1i1p1f1
84,ACCESS-CM2,r5i1p1f1
47,ACCESS-CM2,r10i1p1f1
49,ACCESS-CM2,r7i1p1f1
51,ACCESS-CM2,r9i1p1f1
54,ACCESS-CM2,r8i1p1f1


,source_id,member_id
0,ACCESS-ESM1-5,r1i1p1f1
62,ACCESS-ESM1-5,r22i1p1f1
61,ACCESS-ESM1-5,r11i1p1f1
60,ACCESS-ESM1-5,r24i1p1f1
59,ACCESS-ESM1-5,r26i1p1f1
...,...,...
27,ACCESS-ESM1-5,r32i1p1f1
26,ACCESS-ESM1-5,r15i1p1f2
25,ACCESS-ESM1-5,r2i1p1f1
24,ACCESS-ESM1-5,r15i1p1f1


,source_id,member_id
90,ACCESS-OM2,r1i1p1f1


,source_id,member_id
91,ACCESS-OM2-025,r1i1p1f1


""


In [16]:
ensemble = "r1i1p1f1"

In [17]:
year_start = 1990
num_years = 10

In [18]:
def variable_availability_check(cat, **kwargs):
    """
    List the relevant variables available and if more is needed.
    """
    searched_cat = cat.search(**kwargs)
    umo_cat = searched_cat.search(variable_id = "umo")
    vmo_cat = searched_cat.search(variable_id = "vmo")
    mlotstmo_cat = searched_cat.search(variable_id = "mlotst")

    print("\numo:\n")
    print(available_time_window(umo_cat))
    print("\nvmo:\n")
    print(available_time_window(vmo_cat))
    print("\nmlotst:\n")
    print(available_time_window(mlotstmo_cat))

    return


def available_time_window(cat):
    time_ranges = cat.df.time_range.unique()
    idx = np.argsort([int(foo[0:4]) for foo in time_ranges if foo != 'na'])
    # return time_ranges[idx]
    return time_ranges[idx]

In [19]:
model, experiment, ensemble

('ACCESS-ESM1-5', 'historical', 'r1i1p1f1')

In [20]:
variable_availability_check(cat,
    source_id = model,
    experiment_id = experiment,
    member_id = ensemble,    
    realm = 'ocean'
)                            


umo:

['185001-185912' '186001-186912' '187001-187912' '188001-188912'
 '189001-189912' '190001-190912' '191001-191912' '192001-192912'
 '193001-193912' '194001-194912' '195001-195912' '196001-196912'
 '197001-197912' '198001-198912' '199001-199912' '200001-200912'
 '201001-201412']

vmo:

['185001-185912' '186001-186912' '187001-187912' '188001-188912'
 '189001-189912' '190001-190912' '191001-191912' '192001-192912'
 '193001-193912' '194001-194912' '195001-195912' '196001-196912'
 '197001-197912' '198001-198912' '199001-199912' '200001-200912'
 '201001-201412']

mlotst:

['185001-201412']


In [21]:
# Check that catalog contains the data requested before creating empty directories
searched_cat = cat.search(
    source_id = model,
    experiment_id = experiment,
    member_id = ensemble,
    variable_id = ["umo", "vmo", "mlotst"],
    realm = 'ocean')
np.sort(searched_cat.df.source_id.unique())

array(['ACCESS-ESM1-5'], dtype=object)

In [22]:
searched_searched_cat = searched_cat.search(variable_id = "uo")
searched_searched_cat.df.variable_id.unique()

array([], dtype=object)

In [23]:
np.sort([int(foo[0:4]) for foo in searched_cat.df.time_range.unique() if foo != 'na'])

array([1850, 1850, 1860, 1870, 1880, 1890, 1900, 1910, 1920, 1930, 1940,
       1950, 1960, 1970, 1980, 1990, 2000, 2010])

In [28]:
# Create directory on gdata
datadir = '/scratch/xv83/TMIP/data'
start_time, end_time = time_window_strings(year_start, num_years)
start_time_str = start_time.strftime("%b%Y")
end_time_str = end_time.strftime("%b%Y")

outputdir = f'{datadir}/{model}/{experiment}/{ensemble}/{start_time_str}-{end_time_str}'
print(outputdir)

/scratch/xv83/TMIP/data/ACCESS-ESM1-5/historical/r1i1p1f1/Jan1990-Dec1999


In [29]:
makedirs(outputdir, exist_ok=True)

In [30]:
########## Start the client and make the `.nc` files ##########
print("Starting client")
client = Client(n_workers=4)#, threads_per_worker=1, memory_limit='16GB') # Note: with 1thread/worker cannot plot thetao. Maybe I need to understand why?
client

Starting client


In [31]:
# umo dataset
print("Loading umo data")
umo_datadask = select_latest_data(searched_cat,
    dict(
        chunks={'i': 60, 'j': 60, 'time': -1, 'lev':50}
    ),
    variable_id = "umo",
    frequency = "mon",
)
print("\numo_datadask: ", umo_datadask)

Loading umo data

umo_datadask:  <xarray.Dataset> Size: 43GB
Dimensions:             (time: 1980, lev: 50, j: 300, i: 360, bnds: 2,
                         vertices: 4)
Coordinates:
  * time                (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 20...
    time_bnds           (time, bnds) datetime64[ns] 32kB dask.array<chunksize=(120, 2), meta=np.ndarray>
  * lev                 (lev) float64 400B 5.0 15.0 25.0 ... 5.499e+03 5.831e+03
    lev_bnds            (lev, bnds) float64 800B dask.array<chunksize=(50, 2), meta=np.ndarray>
  * j                   (j) int32 1kB 0 1 2 3 4 5 6 ... 294 295 296 297 298 299
  * i                   (i) int32 1kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
    latitude            (j, i) float64 864kB dask.array<chunksize=(60, 60), meta=np.ndarray>
    longitude           (j, i) float64 864kB dask.array<chunksize=(60, 60), meta=np.ndarray>
    vertices_latitude   (j, i, vertices) float64 3MB dask.array<chunksize=(60, 60, 2), meta=np.ndarray>
    

In [32]:
# vmo dataset
print("Loading vmo data")
vmo_datadask = select_latest_data(searched_cat,
    dict(
        chunks={'i': 60, 'j': 60, 'time': -1, 'lev':50}
    ),
    variable_id = "vmo",
    frequency = "mon",
)
print("\nvmo_datadask: ", vmo_datadask)

Loading vmo data

vmo_datadask:  <xarray.Dataset> Size: 43GB
Dimensions:             (time: 1980, lev: 50, j: 300, i: 360, bnds: 2,
                         vertices: 4)
Coordinates:
  * time                (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 20...
    time_bnds           (time, bnds) datetime64[ns] 32kB dask.array<chunksize=(120, 2), meta=np.ndarray>
  * lev                 (lev) float64 400B 5.0 15.0 25.0 ... 5.499e+03 5.831e+03
    lev_bnds            (lev, bnds) float64 800B dask.array<chunksize=(50, 2), meta=np.ndarray>
  * j                   (j) int32 1kB 0 1 2 3 4 5 6 ... 294 295 296 297 298 299
  * i                   (i) int32 1kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
    latitude            (j, i) float64 864kB dask.array<chunksize=(60, 60), meta=np.ndarray>
    longitude           (j, i) float64 864kB dask.array<chunksize=(60, 60), meta=np.ndarray>
    vertices_latitude   (j, i, vertices) float64 3MB dask.array<chunksize=(60, 60, 2), meta=np.ndarray>
    

In [33]:
# mlotst dataset
print("Loading mlotst data")
mlotst_datadask = select_latest_data(searched_cat,
    dict(
        chunks={'i': 60, 'j': 60, 'time': -1, 'lev':50}
    ),
    variable_id = "mlotst",
    frequency = "mon",
)
print("\nmlotst_datadask: ", mlotst_datadask)

Loading mlotst data

mlotst_datadask:  <xarray.Dataset> Size: 864MB
Dimensions:             (time: 1980, bnds: 2, j: 300, i: 360, vertices: 4)
Coordinates:
  * time                (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 20...
  * j                   (j) int32 1kB 0 1 2 3 4 5 6 ... 294 295 296 297 298 299
  * i                   (i) int32 1kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
    latitude            (j, i) float64 864kB dask.array<chunksize=(60, 60), meta=np.ndarray>
    longitude           (j, i) float64 864kB dask.array<chunksize=(60, 60), meta=np.ndarray>
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds           (time, bnds) datetime64[ns] 32kB dask.array<chunksize=(1980, 2), meta=np.ndarray>
    vertices_latitude   (j, i, vertices) float64 3MB dask.array<chunksize=(60, 60, 2), meta=np.ndarray>
    vertices_longitude  (j, i, vertices) float64 3MB dask.array<chunksize=(60, 60, 2), meta=np.ndarray>
    mlotst              (time, j, i) flo

In [34]:
# Deal with thkcello for a different script,
# given that its location (fixed or time-dependent) depends on the model and/or project
# # thkcello dataset
# print("Loading thkcello data")
# thkcello_datadask = select_latest_data(searched_cat,
#     dict(
#         chunks={'i': 60, 'j': 60, 'time': -1, 'lev':50}
#     ),
#     variable_id = "thkcello",
#     frequency = "mon",
# )
# print("\nthkcello_datadask: ", thkcello_datadask)

In [35]:
# Slice umo dataset for the time period
umo_datadask_sel = umo_datadask.sel(time=slice(start_time, end_time))
# Take the time average of the monthly evaporation (using month length as weights)
umo = umo_datadask_sel["umo"].weighted(umo_datadask_sel.time.dt.days_in_month).mean(dim="time")
umo

<xarray.DataArray 'umo' (lev: 50, j: 300, i: 360)> Size: 43MB
dask.array<truediv, shape=(50, 300, 360), dtype=float64, chunksize=(50, 60, 60), chunktype=numpy.ndarray>
Coordinates:
  * lev        (lev) float64 400B 5.0 15.0 25.0 ... 5.499e+03 5.831e+03
  * j          (j) int32 1kB 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
  * i          (i) int32 1kB 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
    latitude   (j, i) float64 864kB dask.array<chunksize=(60, 60), meta=np.ndarray>
    longitude  (j, i) float64 864kB dask.array<chunksize=(60, 60), meta=np.ndarray>

In [36]:
# Slice vmo dataset for the time period
vmo_datadask_sel = vmo_datadask.sel(time=slice(start_time, end_time))
# Take the time average of the monthly evaporation (using month length as weights)
vmo = vmo_datadask_sel["vmo"].weighted(vmo_datadask_sel.time.dt.days_in_month).mean(dim="time")
vmo

<xarray.DataArray 'vmo' (lev: 50, j: 300, i: 360)> Size: 43MB
dask.array<truediv, shape=(50, 300, 360), dtype=float64, chunksize=(50, 60, 60), chunktype=numpy.ndarray>
Coordinates:
  * lev        (lev) float64 400B 5.0 15.0 25.0 ... 5.499e+03 5.831e+03
  * j          (j) int32 1kB 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
  * i          (i) int32 1kB 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
    latitude   (j, i) float64 864kB dask.array<chunksize=(60, 60), meta=np.ndarray>
    longitude  (j, i) float64 864kB dask.array<chunksize=(60, 60), meta=np.ndarray>

In [37]:
# Slice mlotst dataset for the time period
mlotst_datadask_sel = mlotst_datadask.sel(time=slice(start_time, end_time))
# Take the time mean of the yearly maximum of mlotst
mlotst_yearlymax = mlotst_datadask_sel.groupby("time.year").max(dim="time")
mlotst_yearlymax

<xarray.Dataset> Size: 75MB
Dimensions:             (year: 10, bnds: 2, j: 300, i: 360, vertices: 4)
Coordinates:
  * j                   (j) int32 1kB 0 1 2 3 4 5 6 ... 294 295 296 297 298 299
  * i                   (i) int32 1kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
    latitude            (j, i) float64 864kB dask.array<chunksize=(60, 60), meta=np.ndarray>
    longitude           (j, i) float64 864kB dask.array<chunksize=(60, 60), meta=np.ndarray>
  * year                (year) int64 80B 1990 1991 1992 1993 ... 1997 1998 1999
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds           (year, bnds) datetime64[ns] 160B dask.array<chunksize=(10, 2), meta=np.ndarray>
    mlotst              (year, j, i) float32 4MB dask.array<chunksize=(10, 60, 60), meta=np.ndarray>
    vertices_latitude   (year, j, i, vertices) float64 35MB dask.array<chunksize=(10, 60, 60, 2), meta=np.ndarray>
    vertices_longitude  (year, j, i, vertices) float64 35MB dask.array<chunksize=(10, 60, 60, 2), meta=np.ndarray>
Attributes: (12/64)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            21915.0
    creation_date:                    2019-11-15T15:38:06Z
    ...                               ...
    intake_esm_attrs:variable_id:     mlotst
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:version:         v20191115
    intake_esm_attrs:time_range:      185001-201412
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r...

In [38]:
mlotst = mlotst_yearlymax.mean(dim="year")
mlotst

<xarray.Dataset> Size: 9MB
Dimensions:             (j: 300, i: 360, vertices: 4)
Coordinates:
  * j                   (j) int32 1kB 0 1 2 3 4 5 6 ... 294 295 296 297 298 299
  * i                   (i) int32 1kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
    latitude            (j, i) float64 864kB dask.array<chunksize=(60, 60), meta=np.ndarray>
    longitude           (j, i) float64 864kB dask.array<chunksize=(60, 60), meta=np.ndarray>
Dimensions without coordinates: vertices
Data variables:
    mlotst              (j, i) float32 432kB dask.array<chunksize=(60, 60), meta=np.ndarray>
    vertices_latitude   (j, i, vertices) float64 3MB dask.array<chunksize=(60, 60, 2), meta=np.ndarray>
    vertices_longitude  (j, i, vertices) float64 3MB dask.array<chunksize=(60, 60, 2), meta=np.ndarray>

In [39]:
# # Slice thkcello dataset for the time period
# thkcello_datadask_sel = thkcello_datadask.sel(time=slice(start_time, end_time))
# # Take the time average of the monthly evaporation (using month length as weights)
# thkcello = thkcello_datadask_sel["thkcello"].weighted(thkcello_datadask_sel.time.dt.days_in_month).mean(dim="time")

In [40]:
# Save to netcdfs (and compute!)
umo.to_netcdf(f'{outputdir}/umo.nc', compute=True)
vmo.to_netcdf(f'{outputdir}/vmo.nc', compute=True)
mlotst.to_netcdf(f'{outputdir}/mlotst.nc', compute=True)
# thkcello.to_netcdf(f'{outputdir}/thkcello.nc', compute=True)

In [41]:
client.close()